In [1]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [2]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [3]:
# initialize models

openai = OpenAI()
OPENAI_MODEL = "gpt-4o"


In [18]:
### Define System Message

system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [5]:
### Define User Prompt

def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [6]:
### Roles

def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [7]:
### Create function to call openai api to convert code and yield result

def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [9]:
### Created for Gradio UI

def optimize(python):

    result = stream_gpt(python)
    for stream_so_far in result:
        yield stream_so_far        

In [16]:
###Creating file for c++ code locally

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [11]:
### Sample Python code from gfg

python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [21]:
### Function for Gradio UI to execute Python Code generated

def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
        sys.stdout = sys.__stdout__
        return output.getvalue()
    except Exception as e:
        sys.stdout = sys.__stdout__
        return f"An error occurred in Python execution:\n{e}"

In [22]:
### Function for Gradio UI to execute the C++ code generated

def execute_cpp(code):
        write_output(code)
        try:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "optimized", "optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [30]:
### Simple UI for


css = """
.python {background-color: #f0f8ff; border-radius: 5px; padding: 10px;}
.cpp {background-color: #f5f5dc; border-radius: 5px; padding: 10px;}
.button {background-color: #4CAF50; color: white; padding: 10px 20px; border: none; border-radius: 5px; cursor: pointer;}
.button:hover {background-color: #45a049;}
"""



with gr.Blocks(css=css) as ui:
    gr.Markdown(
        """
        ## Python to C++ Code Conversion Tool 🐍➡️➕➕
        Easily convert Python code into its C++ equivalent, execute the code, and view the results.
        """
    )
    with gr.Row():
        with gr.Column():
            python = gr.Textbox(
                label="Python Code",
                placeholder="Enter your Python code here...",
                lines=12,
                elem_classes=["python"],
            )
        with gr.Column():
            cpp = gr.Textbox(
                label="C++ Code",
                placeholder="Converted C++ code will appear here...",
                lines=12,
                elem_classes=["cpp"],
            )
    with gr.Row():
        convert = gr.Button("Convert Code", elem_classes=["button"])
    with gr.Row():
        with gr.Column():
            python_run = gr.Button("Run Python Code", elem_classes=["button"])
            python_out = gr.TextArea(
                label="Python Execution Result",
                placeholder="Python code output will appear here...",
                elem_classes=["python"],
            )
        with gr.Column():
            cpp_run = gr.Button("Run C++ Code", elem_classes=["button"])
            cpp_out = gr.TextArea(
                label="C++ Execution Result",
                placeholder="C++ code output will appear here...",
                elem_classes=["cpp"],
            )
    
    # Define interactivity
    convert.click(optimize, inputs=[python], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)
